In [1]:
from draftfast import rules
from draftfast.optimize import run, run_multi
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.lineup_constraints import LineupConstraints

import pandas as pd
import sqlite3

db_path = './fantasy_nfl.db'
table_name = 'player_proj'

In [4]:
con = sqlite3.connect(db_path)
q = "SELECT * FROM '{}' WHERE pull_datetime=(SELECT MAX(pull_datetime) FROM '{}')"
df = pd.read_sql(con=con, sql=q.format(table_name,table_name))
con.close()

# del df['index']

In [5]:
matchups_df = df.groupby(['team','Opponent Team']).sum()[[]].reset_index()

matchup_dict = {}
for a,b in matchups_df.values:
    if a > b:
        matchup_dict[a] = '{}@{}'.format(a,b)
    elif a < b:
        matchup_dict[a] = '{}@{}'.format(b,a)

matchups = pd.DataFrame.from_dict(matchup_dict, orient='index', columns=['matchup']).reset_index()

In [6]:
service = 'FanDuel'

In [7]:
defs = pd.read_csv('./defs.csv')
defs['{} Cost'.format(service)] = defs['{} Cost'.format(service)].apply(lambda x: x.replace(',',''))
df = df.append(defs)

In [8]:
df['{} Cost'.format(service)] = df['{} Cost'.format(service)].apply(lambda x: int(x.replace('$','').replace('N/A','0')))

In [9]:
df = df.merge(matchups, left_on=['team'], right_on=['index'], how='left')

In [10]:
del df['index']

In [11]:
df = df[df['{} Cost'.format(service)] > 0]
df = df.reset_index()
del df['index']

In [ ]:
exposure_bounds=[
        {
            'name': 'Damian Lillard',
            'min': 0.3,
            'max': 0.5,
        },
        {
            'name': 'LaMarcus Aldridge',
            'min': 0.5,
            'max': 0.7,
        },
    ],

In [15]:
service = 'FanDuel'
univ_min = 0.0
univ_max = 0.30
player_pool = []
exposure_bounds = []
for i, player in enumerate(df['full_name']):
    player_pool.append(Player(pos=df.loc[i]['pos'],
                              name=player,
                              cost=df.loc[i]['{} Cost'.format(service)],
                              proj=float(df.loc[i]['Standard Fantasy Points']),
                              team=df.loc[i]['team'],
                              matchup=df.loc[i]['matchup'],
                              ))
#     exposure_bounds.append({'name':df.loc[i]['full_name'],
#                            'min':univ_min,
#                            'max':univ_max})

In [ ]:
roster, _ = run_multi(
    rule_set=rules.FD_NFL_RULE_SET,
    player_pool=player_pool,
    verbose=True,
    iterations=20,
    exposure_bounds=exposure_bounds
)

Optimal roster for: NFL
+----------+--------------------+------+---------+--------+------------+----------+--------+
| Position | Player             | Team | Matchup | Salary | Projection | vs. Avg. | Locked |
+----------+--------------------+------+---------+--------+------------+----------+--------+
| QB       | Matthew Stafford   | DET  | JAC@DET |  7,300 |      20.88 |    20.88 |        |
| RB       | Derrick Henry      | TEN  | TEN@HOU |  9,000 |      21.77 |    21.77 |        |
| RB       | Kareem Hunt        | CLE  | PIT@CLE |  7,000 |      19.51 |    19.51 |        |
| RB       | Alexander Mattison | MIN  | MIN@ATL |  7,000 |       19.3 |    19.30 |        |
| WR       | Adam Thielen       | MIN  | MIN@ATL |  7,400 |      17.89 |    17.89 |        |
| WR       | A.J. Brown         | TEN  | TEN@HOU |  6,500 |      15.79 |    15.79 |        |
| WR       | Tyler Boyd         | CIN  | IND@CIN |  6,100 |      14.63 |    14.63 |        |
| TE       | Zach Ertz          | PHI  | PHI@B

Optimal roster for: NFL
+----------+--------------------+------+---------+--------+------------+----------+--------+
| Position | Player             | Team | Matchup | Salary | Projection | vs. Avg. | Locked |
+----------+--------------------+------+---------+--------+------------+----------+--------+
| QB       | Matthew Stafford   | DET  | JAC@DET |  7,300 |      20.88 |    20.88 |        |
| RB       | Kareem Hunt        | CLE  | PIT@CLE |  7,000 |      19.51 |    19.51 |        |
| RB       | Alexander Mattison | MIN  | MIN@ATL |  7,000 |       19.3 |    19.30 |        |
| RB       | Myles Gaskin       | MIA  | NYJ@MIA |  5,700 |      16.38 |    16.38 |        |
| WR       | Calvin Ridley      | ATL  | MIN@ATL |  8,600 |      19.09 |    19.09 |        |
| WR       | Adam Thielen       | MIN  | MIN@ATL |  7,400 |      17.89 |    17.89 |        |
| WR       | Terry McLaurin     | WSH  | WSH@NYG |  6,900 |      16.48 |    16.48 |        |
| TE       | Zach Ertz          | PHI  | PHI@B

Optimal roster for: NFL
+----------+--------------------+------+---------+--------+------------+----------+--------+
| Position | Player             | Team | Matchup | Salary | Projection | vs. Avg. | Locked |
+----------+--------------------+------+---------+--------+------------+----------+--------+
| QB       | Matthew Stafford   | DET  | JAC@DET |  7,300 |      20.88 |    20.88 |        |
| RB       | Mike Davis         | CAR  | CHI@CAR |  7,500 |      18.75 |    18.75 |        |
| RB       | Kareem Hunt        | CLE  | PIT@CLE |  7,000 |      19.51 |    19.51 |        |
| RB       | Alexander Mattison | MIN  | MIN@ATL |  7,000 |       19.3 |    19.30 |        |
| WR       | Adam Thielen       | MIN  | MIN@ATL |  7,400 |      17.89 |    17.89 |        |
| WR       | Allen Robinson     | CHI  | CHI@CAR |  7,000 |      17.22 |    17.22 |        |
| WR       | A.J. Brown         | TEN  | TEN@HOU |  6,500 |      15.79 |    15.79 |        |
| TE       | Zach Ertz          | PHI  | PHI@B